# Prokaryotic Promoters

This working prototype that reads a single string, verifies that it is a valid DNA sequence, finds exact matches to common promoter motifs, and reports an overall match score for each proposed promoter.

###  The DNA verification process

DNA normally consists of 4 different nucleotieds **A, T, G, and C.** We can scan the string to check if we find any string other than these characters. Linear Scanning can perform this operation in **O(n)** time.

### Promoter Matching Process

We can locate promoters using an exact matching algorithm. This process is very straightforward however is not always accurate. Fuzzy matching process can locate multiple possible promoter regions. Fuzzy matching technique generates a list of possible promoter regions with their corresponding probability of being a promoter.

##### Exact Matching

For exact promoter matching, Two regions indicating a promoter found together are the  **TTGACA (-35) and the TATAAT (-10)** regions. Which means, we can simply scan for these two regions to locate the promoter start regions. 

We can keep track of how spread apart **TTGACA(-35) and TATAAT (-10)** are, making sure their **distance (d)** $\le$ 21 and $\ge$ 15

To make things simple, for exact matching we will be looking for **TTGACA** and **TATAAT** or **TAXXXT** blocks.

To calculate the overall confidence score:
    
- We can use the normal distribution formula constrained between the range [21,15] where $\mu = 17$, and $\sigma = 0.8$.
- **TODO: Explain character matching loss methodology**

##### Fuzzy Matching

These are the conditions to keep in mind: 
- In the TATAAT region, TAXXXT are conserved over 80% of the time (X means less conserved)

- In the TTGACA region of the first 3 (TTG) at least one is always conserved but 2 and most often 3 are usually conserved.

- Both units tend to occcur about 17 basepairs away from eachother, but this can vary from as little as 15 to as high as 21 (however 95% of the time it is betwren 16 and 18, with more than half of that being 17)

- The -35 region is further upstream from the -10 (-35 and -10 describe the base pair distance from the starting region)